In [49]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as st
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 18, 6
import seaborn as sns
import sklearn
from sklearn.ensemble.forest import RandomForestRegressor
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import operator 
import psycopg2
import math
import requests
import json
from flask import jsonify
import csv
from csv import writer
from statistics import mean
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from numpy import cov
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from numpy import mean
from numpy import std
import statistics

In [50]:

url_new_pref = 'https://covid19-api.org/api/status?date=2020-08-13'
url_iso2 = 'https://restcountries.eu/rest/v2/'


response_iso2 = requests.get(url_iso2)
lst_iso2 = json.loads(response_iso2.text)
dict_name = {}
dict_iso2 = {}
dict_timeline = {}

responses = []

latestaffected={}
latestdeath={}
latestrecovered={}
population={}



In [51]:
with open('countrylatestactive.csv', 'w', encoding='utf-8') as csv_file:
    csv_writer = writer(csv_file)
    headers = ['Country_name', 'Country_code_iso2','population','Latest_date','Tot_affected','Tot_dead','Tot_recov','Affected per thousand','Active']
    csv_writer.writerow(headers)
    for i in lst_iso2:
        # print(i['name'], i['population'])
        # dict_name[i['alpha2Code']] = i['name']
        # dict_iso3[i['alpha2Code']] = i['alpha3Code']
        dict_iso2[i['alpha2Code']] = i['alpha2Code']
        dict_name[i['alpha2Code']] = i['name']
        population[i['alpha2Code']] = i['population']
    res = requests.get(url_new_pref)
    lst_res = json.loads(res.text)
    for j in lst_res:
        print(j)
        code = j['country']
        print(dict_name[code])
        print(code)
        print(dict_iso2[code])
        entry = [dict_name[code], dict_iso2[code], population[code], j['last_update'].split('T')[0], j['cases'], j['deaths'], j['recovered'],(j['cases']/population[code]) * 1000, j['cases']-(j['recovered']+ j['deaths'])]
        csv_writer.writerow(entry)
#     for i in lst['confirmed']['locations']:
#         name = i['country']
#         code = i['country_code']
#         if code in dict_iso3:
#             active = latestaffected[code]-(latestrecovered[code]+ latestdeath[code])
#             entry = [name, code,population[code],latestdate,latestaffected[code],latestdeath[code],latestrecovered[code],(latestaffected[code]/population[code]) * 1000, active ,(active/latestaffected[code])*100,(active/(latestrecovered[code]+active))*100]
#             csv_writer.writerow(entry)


{'country': 'US', 'last_update': '2020-08-12T23:27:29', 'cases': 5191689, 'deaths': 165909, 'recovered': 1753760}
United States of America
US
US
{'country': 'BR', 'last_update': '2020-08-12T23:27:29', 'cases': 3109630, 'deaths': 103026, 'recovered': 2473376}
Brazil
BR
BR
{'country': 'IN', 'last_update': '2020-08-12T23:27:29', 'cases': 2329638, 'deaths': 46091, 'recovered': 1639599}
India
IN
IN
{'country': 'RU', 'last_update': '2020-08-12T23:27:29', 'cases': 900745, 'deaths': 15231, 'recovered': 708900}
Russian Federation
RU
RU
{'country': 'ZA', 'last_update': '2020-08-12T23:27:29', 'cases': 568919, 'deaths': 11010, 'recovered': 432029}
South Africa
ZA
ZA
{'country': 'MX', 'last_update': '2020-08-12T23:27:29', 'cases': 492522, 'deaths': 53929, 'recovered': 397278}
Mexico
MX
MX
{'country': 'PE', 'last_update': '2020-08-12T23:27:29', 'cases': 489680, 'deaths': 21501, 'recovered': 329404}
Peru
PE
PE
{'country': 'CO', 'last_update': '2020-08-12T23:27:29', 'cases': 410453, 'deaths': 13475, '

Deleting duplicates

In [52]:
with open('countrylatestactive.csv','r') as in_file, open('countrylatestactive2.csv','w') as out_file:
    seen = set() # set for fast O(1) amortized lookup
    for line in in_file:
        if line in seen: continue # skip duplicate

        seen.add(line)
        out_file.write(line)

Fetching latest data of pollution and health from APi


In [53]:
url1 = 'https://www.numbeo.com/api/rankings_by_country_historical?api_key=r8umol7u42vvbs&section=12&fbclid=IwAR2g23utsR5KC5PvEU91RyQk8Z3DWno1BMqEL-KsbgAG43aQc-RxX4TDN0w'
response1 = requests.get(url1)
lst1 = json.loads(response1.text)

health_index = {}


for i in lst1['2020']:
    
    health_index[i['iso3166_country_code']] = i['healthcare_index']
    
url2 = 'https://www.numbeo.com/api/rankings_by_country_historical?api_key=r8umol7u42vvbs&section=8&fbclid=IwAR2g23utsR5KC5PvEU91RyQk8Z3DWno1BMqEL-KsbgAG43aQc-RxX4TDN0w'
response2 = requests.get(url2)
lst2 = json.loads(response2.text)

pollution_index = {}


for i in lst2['2020']:
    
    pollution_index[i['iso3166_country_code']] = i['pollution_index']

In [54]:
with open('global_health_index_ranking_comma.csv', 'w') as csv_file:
    csv_writer = writer(csv_file)
    headers = ['country name', 'Country_code_iso2','healthcare index']
    csv_writer.writerow(headers)
    for i in lst1['2020']:
        name = i['country']
        code = i['iso3166_country_code']
        entry = [name, code,health_index[code]]
        csv_writer.writerow(entry)
        
with open('global_pollution_index_ranking_comma.csv', 'w') as csv_file:
    csv_writer = writer(csv_file)
    headers = ['country name', 'Country_code_iso2','pollution index']
    csv_writer.writerow(headers)
    for i in lst2['2020']:
        name = i['country']
        code = i['iso3166_country_code']
        entry = [name, code,pollution_index[code]]
        csv_writer.writerow(entry)

Making csv containg number of tests

In [55]:



url3 = 'https://disease.sh/v2/countries'

response3 = requests.get(url3)
lst3 = json.loads(response3.text)

dict_name = {}
population = {}
dict_iso3 = {}
dict_noOfTests = {}
total_cases = {}
total_deaths = {}
total_recovered = {}
live_cases = {}
live_deaths = {}
live_recovered = {}
active = {}
critical = {}
casesPerOneMillion = {}
deathsPerOneMillion = {}
testsPerOneMillion = {}
recoveredPerOneMillion = {}
criticalPerOneMillion = {}
continent = {}


with open('country_populationtests.csv','w',encoding="utf-8") as csv_file:
    csv_writer = writer(csv_file)
    headers = ['Countryname', 'Countrycodeiso2', 'Countrycodeiso3','Continent','Population','Totaltests','Totalaffected','Totaldeaths','Totalrecovered','Active','Critical','Deathrate','Tests per million','Affected per million','Deaths per million','Recovered per million']
    csv_writer.writerow(headers)
    for i in lst3:
        name = i['country']
        iso2 = i['countryInfo']['iso2']
        iso3 = i['countryInfo']['iso3']
        noOfTests = i['tests']
        population = i['population']
        total_cases = i['cases']
        total_deaths = i['deaths']
        total_recovered =i['recovered']
        active = i['active']
        critical = i['critical']
        casesPerOneMillion = i['casesPerOneMillion']
        deathsPerOneMillion = i['deathsPerOneMillion']
        testsPerOneMillion = i['testsPerOneMillion']
        recoveredPerOneMillion = i['recoveredPerOneMillion']
        criticalPerOneMillion = i['criticalPerOneMillion']
        continent = i['continent']
        
        entry = [name, iso2, iso3,continent,population,noOfTests,total_cases,total_deaths,total_recovered,active,critical,(total_deaths/total_cases)*100,testsPerOneMillion,casesPerOneMillion,deathsPerOneMillion,recoveredPerOneMillion]
        csv_writer.writerow(entry)
    




Doing some data cleaning (0 deaths, <100 affected, 0 tests and tests< affected are removed

In [56]:
data = pd.read_csv('country_populationtests.csv')
finaldata = data[(data.Totaltests>0) & (data.Totalaffected>=100) & (data.Totaldeaths>0) & (data.Totaltests>=data.Totalaffected)]
header = ["Countryname", "Population", "Totaltests", "Deathrate"]
finaldata.to_csv('deathrate+tests.csv', columns = header,index=False)

Joining two csv files (final csv is combinedfood.csv)


In [57]:
df2 = pd.read_csv('global_food_index_ranking_comma.csv')
df1 = pd.read_csv('countrylatestactive2.csv')
res1 = pd.merge(df2, df1,on ='Country_code_iso2',how='left')
res1.to_csv('combinedfood.csv', index=False)
df3 = pd.read_csv('global_health_index_ranking_comma.csv')
res2 = pd.merge(df3, df1,on ='Country_code_iso2',how='left')
res2.to_csv('combinedhealth.csv', index=False)
df4 = pd.read_csv('global_pollution_index_ranking_comma.csv')
res3 = pd.merge(df4, df1,on ='Country_code_iso2',how='left')
res3.to_csv('combinedpollution.csv', index=False)

inserting

In [58]:
def insertInfo(date,method,independant,dependant,corelcoeff):
    try:
        postgres_insert_query = """INSERT INTO crcdata(date,method,independant,dependant,corelcoeff)VALUES(%s,%s,%s,%s,%s)"""
        cursor.execute(postgres_insert_query, (date,method,independant,dependant,corelcoeff))

        connection.commit()
        count = cursor.rowcount
        print (count, "Record inserted successfully into crcdata table")
        
    except (Exception, psycopg2.Error) as error :
        if(connection):
            print("Error inserting data in PostgreSQL table", error)


correlation

In [59]:

def correl(bd,dependent,independent):
    Bd1 = bd.iloc[:,2:3]
    Bd2 = bd.iloc[:,3:4]
    
    
    X = [x[0] for x in Bd1.values.tolist()]  #list comprehension
    Y = [y[0] for y in Bd2.values.tolist()]
    print('data1: mean=%.3f stdv=%.3f' % (mean(X), std(X)))
    print('data2: mean=%.3f stdv=%.3f' % (mean(Y), std(Y)))
    print('overall food security: median=%.3f' % statistics.median(X))
    print('total affected: median=%.3f' % statistics.median(Y))
    
    
    #print('my function pearson : %.5f' % pearson_def(X,Y))
    
    covariance = cov(X,Y)
    print('covarience value :')
    print( covariance)
    
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=X,y=Y,mode='markers',marker_color=Y)) # hover text goes here

#     fig.update_layout(title='correlations', xaxis=dict(title='X'),yaxis=dict(title='Y'))
    
#     fig.show()
    ##pyplot.scatter(Bd1, Bd2)
    #pyplot.xlabel("temparature")
    #pyplot.ylabel("Total Affected")
    #pyplot.show()
    alpha = 0.05 #setting my significant level 5 %
    corr1, r = pearsonr(X,Y)
    print('Pearsons correlation: %.3f' % corr1)
   
    insertInfo(latestdate,'Pearson',independent,dependent,corr1)
    if r > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % r)
        
    else:
        print('Samples are correlated (reject H0) p=%.3f' % r)
        
        
    corr2, p = spearmanr(X,Y)
    print('Spearmans correlation: %.3f' % corr2)
    
    insertInfo(latestdate,'Spearman',independent,dependent,corr2)
    # interpret the significance
    
    if p > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
        
    else:
        print('Samples are correlated (reject H0) p=%.3f' % p)
       
        
    
    coef, q = kendalltau(X,Y)
    print('Kendall correlation coefficient: %.3f' % coef)
    insertInfo(latestdate,'Kendall',independent,dependent,coef)
    
    # interpret the significance
    alpha = 0.05
    if q > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % q)
        
    else:
        print('Samples are correlated (reject H0) p=%.3f' % q)
        

    


In [60]:
with open('combinedfood.csv','r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    with open('affected+food.csv','w') as new_file1:
        fieldnames1 = ['Country_name','population','Overall','Tot_affected','Latest_date']
        
        csv_writer1 = csv.DictWriter(new_file1, fieldnames= fieldnames1)
        csv_writer1.writeheader()
        
        for line1 in csv_reader:
            del line1['Region']
            del line1['idcount']
            del line1['Country_code_iso2']
            del line1['Country3']
            del line1['Date']
            del line1['Affordability']
            del line1['Availability']
            del line1['Safety']
            del line1['Tot_dead']
            del line1['Tot_recov']
            del line1['Affected per thousand']
            del line1['Active']
            #print(line['CFR1'],line['CFR2'])
            #print(line1)
            csv_writer1.writerow(line1)
            
with open('combinedhealth.csv','r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    with open('recovered+health.csv','w') as new_file1:
        fieldnames1 = ['country name','population','healthcare index','Tot_recov','Latest_date']
        
        csv_writer1 = csv.DictWriter(new_file1, fieldnames= fieldnames1)
        csv_writer1.writeheader()
        
        for line1 in csv_reader:
            del line1['Country_code_iso2']
            del line1['Tot_affected']
            del line1['Tot_dead']
            del line1['Affected per thousand']
            del line1['Active']
            del line1['Country_name']
            #print(line['CFR1'],line['CFR2'])
            #print(line1)
            csv_writer1.writerow(line1)
            
with open('combinedpollution.csv','r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    with open('affected+pollution.csv','w') as new_file1:
        fieldnames1 = ['country name','population','pollution index','Tot_affected','Latest_date']
        
        csv_writer1 = csv.DictWriter(new_file1, fieldnames= fieldnames1)
        csv_writer1.writeheader()
        
        for line1 in csv_reader:
            del line1['Country_code_iso2']
            del line1['Tot_recov']
            del line1['Tot_dead']
            del line1['Affected per thousand']
            del line1['Active']
            del line1['Country_name']
            #print(line['CFR1'],line['CFR2'])
            #print(line1)
            csv_writer1.writerow(line1)

In [61]:
latestdate = '08/13/2020'

final call

In [62]:
import psycopg2
connection = psycopg2.connect(user = "teamcoronosis",
                                  password = "p1jGQwEMANAGZ5b72xWh",
                                  host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
                                  port = "5432",
                                  database = "postgres")
cursor = connection.cursor()

food = pd.read_csv('affected+food.csv')

correl(food,'Total Affected','Food Security Index')
health = pd.read_csv('recovered+health.csv')
health = health[health['country name']!='Hong Kong']
correl(health,'Total Recovered','Heath Index')

pollution = pd.read_csv('affected+pollution.csv')
pollution = pollution[(pollution['country name']!='Puerto Rico') & (pollution['country name']!='Hong Kong')]
correl(pollution,'Total Affected','Pollution Index')
tests = pd.read_csv('deathrate+tests.csv')

correl(tests,'Death Rate','Total Tests')
insertInfo(latestdate,'Pearson','Temperature','Affected / Day',0.0866)
insertInfo(latestdate,'Spearman','Temperature','Affected / Day',0.0930)
insertInfo(latestdate,'Kendall','Temperature','Affected / Day',0.0777)
insertInfo(latestdate,'Pearson','Humidity','Affected / Day',-0.4951)
insertInfo(latestdate,'Spearman','Humidity','Affected / Day',-0.4366)
insertInfo(latestdate,'Kendall','Humidity','Affected / Day',-0.3155)


if(connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")
    

data1: mean=62.888 stdv=13.819
data2: mean=173770.150 stdv=606262.539
overall food security: median=63.800
total affected: median=27938.000
covarience value :
[[1.92676452e+02 1.60504366e+06]
 [1.60504366e+06 3.70836000e+11]]
Pearsons correlation: 0.190
1 Record inserted successfully into crcdata table
Samples are correlated (reject H0) p=0.044
Spearmans correlation: 0.502
1 Record inserted successfully into crcdata table
Samples are correlated (reject H0) p=0.000
Kendall correlation coefficient: 0.334
1 Record inserted successfully into crcdata table
Samples are correlated (reject H0) p=0.000
data1: mean=64.825 stdv=9.839
data2: mean=150948.013 stdv=385684.715
overall food security: median=65.917
total affected: median=32700.000
covarience value :
[[ 9.80548874e+01 -2.36891763e+05]
 [-2.36891763e+05  1.50659785e+11]]
Pearsons correlation: -0.062
1 Record inserted successfully into crcdata table
Samples are uncorrelated (fail to reject H0) p=0.589
Spearmans correlation: -0.059
1 Record

In [22]:
def deleteData(date):
    try:
        connection = psycopg2.connect(user = "teamcoronosis",
                                  password = "p1jGQwEMANAGZ5b72xWh",
                                  host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
                                  port = "5432",
                                  database = "postgres")
        cursor = connection.cursor()

        # Update single record now
        sql_delete_query = """Delete from crcdata where date = %s"""
        cursor.execute(sql_delete_query, (date, ))
        connection.commit()
        count = cursor.rowcount
        print(count, "Record deleted successfully ")

    except (Exception, psycopg2.Error) as error:
        print("Error in Delete operation", error)

    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [23]:
deleteData('2020/07/08')

18 Record deleted successfully 
PostgreSQL connection is closed


In [2]:
def updateData(af):
    try:
        connection = psycopg2.connect(user = "teamcoronosis",
                                  password = "p1jGQwEMANAGZ5b72xWh",
                                  host = "coronosis.cfzyz28p01p7.ap-south-1.rds.amazonaws.com",
                                  port = "5432",
                                  database = "postgres")
        cursor = connection.cursor()

        # Update single record now
        sql_delete_query = """update crcdata
set dependant = 'Affected / Day'
where dependant = %s"""
        cursor.execute(sql_delete_query, (af, ))
        connection.commit()
        count = cursor.rowcount
        print(count, "Record updated successfully ")

    except (Exception, psycopg2.Error) as error:
        print("Error in update operation", error)

    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

In [3]:
updateData('Affected')

6 Record updated successfully 
PostgreSQL connection is closed
